Кирилл Лалаянц, R33352

# Лабораторная работа No5
## Типовые динамические звенья

Вариант 6

Импорт необходимых для работы библиотек. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import control 
import sympy
import os
from tbcontrol.symbolic import routh
import math
import tqdm

SAVE_PATH = 'tex-report/figs/'
os.makedirs(SAVE_PATH, exist_ok=True)

sympy.init_printing()
p = sympy.Symbol("p")
s = sympy.Symbol("s")
t = sympy.Symbol("t")
w = sympy.Symbol("w")
I = sympy.I


def get_t(end_t = 10, dt=0.001, start_t = 0):
    return np.linspace(start_t, end_t, int(end_t / dt))

## Task 1

In [ ]:
def plot_bode(ss, theory_lambda, freqs, plot_name, save_name, fsize=40):
    mag, phase, omega = control.bode(ss, freqs, plot=False)
    freqs = np.array(freqs)
    theory = theory_lambda(freqs)
    mag_theory = np.absolute(theory)
    phase_theory = np.angle(theory)
    
    
    fig, axs = plt.subplots(2, 2, figsize=(20, 20))
    fig.suptitle(plot_name, fontsize=fsize , y=0.95)
    axs[0, 0].plot(omega, mag, linewidth=12, label='experiment')
    axs[0, 0].plot(omega, mag_theory, 'r--', linewidth=4, label='theory')
    axs[0, 0].set_title('AЧХ', fontsize=fsize)
    axs[0, 0].grid(True)
    axs[0, 0].set_xlabel(f"Частота, [рад/c]", fontsize=fsize // 2)
    axs[0, 0].set_ylabel(f"Амплитуда", fontsize=fsize // 2)
    axs[0, 0].tick_params(axis='both', which='major', labelsize=fsize // 2)
    axs[0, 0].tick_params(axis='both', which='minor', labelsize=fsize // 2)
    axs[0, 0].legend(fontsize=20)
    
    
    axs[0, 1].plot(omega, phase, linewidth=12, label='experiment')
    axs[0, 1].plot(omega, phase_theory, 'r--', linewidth=4, label='theory')
    axs[0, 1].set_title('ФЧХ', fontsize=fsize)
    axs[0, 1].grid(True)
    axs[0, 1].set_xlabel(f"Частота, [рад/c]", fontsize=fsize // 2)
    axs[0, 1].set_ylabel(f"Фаза, [рад]", fontsize=fsize // 2)
    axs[0, 1].tick_params(axis='both', which='major', labelsize=fsize // 2)
    axs[0, 1].tick_params(axis='both', which='minor', labelsize=fsize // 2)
    axs[0, 1].legend(fontsize=20)    
    
    axs[1, 0].plot(omega, mag, linewidth=12, label='experiment')
    axs[1, 0].plot(omega, mag_theory, 'r--', linewidth=4, label='theory')
    axs[1, 0].set_title('ЛAЧХ', fontsize=fsize)
    axs[1, 0].set_xscale('log')
    axs[1, 0].set_yscale('log')
    axs[1, 0].grid(True, which="both", ls="-", color='0.65')
    axs[1, 0].set_xlabel(f"Частота, [рад/c]", fontsize=fsize // 2)
    axs[1, 0].set_ylabel(f"Амплитуда, [дБ]", fontsize=fsize // 2)
    axs[1, 0].tick_params(axis='both', which='major', labelsize=fsize // 2)
    axs[1, 0].tick_params(axis='both', which='minor', labelsize=fsize // 2)  
    axs[1, 0].legend(fontsize=20)  
    
    axs[1, 1].plot(omega, phase, linewidth=12, label='experiment')
    axs[1, 1].plot(omega, phase_theory, 'r--', linewidth=4, label='theory')
    axs[1, 1].set_title('ЛФЧХ', fontsize=fsize)
    axs[1, 1].set_xscale('log')
    axs[1, 1].grid(True, which="both", ls="-", color='0.65')
    axs[1, 1].set_xlabel(f"Частота, [рад/c]", fontsize=fsize // 2)
    axs[1, 1].set_ylabel(f"Фаза, [рад]", fontsize=fsize // 2)
    axs[1, 1].tick_params(axis='both', which='major', labelsize=fsize // 2)
    axs[1, 1].tick_params(axis='both', which='minor', labelsize=fsize // 2)
    axs[1, 1].legend(fontsize=20)
    
    plt.savefig(f'{SAVE_PATH}/{save_name}.jpg') 
    
def get_ss_and_theory(subs, sA, sB, sC, sD):
    stf = (sC @ (sympy.diag([p] * 2, unpack=True) - sA).inv() @ sB + sD)[0]
    wjw = stf.subs({p: I*w}).subs(subs)
    theory = sympy.lambdify(w, wjw, 'numpy')
    
    sympy_to_numpy = lambda mat, subs: np.array(mat.subs(subs)).astype(np.float64)
    A = sympy_to_numpy(sA, subs)
    B = sympy_to_numpy(sB, subs)
    C = sympy_to_numpy(sC, subs)
    D = sympy_to_numpy(sD, subs)
    
    return control.ss(A, B, C, D), theory


### 2. Brushed DC motor 2.0. (v3)

In [ ]:
t2_K_M, t2_K_E, t2_J, t2_R, t2_L = 0.3612, 0.3612, 0.0031, 4.7237, 1.0567
k_m, k_e, J, R, L = sympy.symbols('k_m k_e J R L')
t2_subs = {k_m: t2_K_M, k_e: t2_K_E, J: t2_J, R: t2_R, L: t2_L}

t2_sA = sympy.Matrix([[0, 1 / J], [-k_m * k_e / L, -R / L]])
t2_sB = sympy.Matrix([0, k_m / L])
t2_sC = sympy.Matrix([[1, 0]])
t2_sD = sympy.Matrix([0])

t2_ss, t2_theory = get_ss_and_theory(t2_subs, t2_sA, t2_sB, t2_sC, t2_sD)

plot_bode(t2_ss, t2_theory, [i/10 for i in range(1000)], 'Brushed DC motor 2.0.', '1_2')